# Wide World Importers - Marketing Experimentation Analysis Preparation - Jan 2016

_Last updated: Feb 2, 2016_ \
_Author(s): Julie Koesmarno_ 

## Abstract

Wide World Importers CEO is keen to understand the revenue, profit and margin spread of their products. This paper is to prepare for the next Marketing Planning Meeting with marketing campaign as one of the topics. They would like to experiment and strategize on marketing campaign. The goal of this document is to identify the **two distinct product groups** based on predefined agreed rules for the experiments. Stock items belonging to the identified Product Groups will be subject to independent A/B marketing campaign experiements. 

## Introduction 
This document will use the Wide World Importers database to do analysis on product portfolio in the past month. We would like to identify the two product groups for planned experimentation in the next Marketing Planning Meeting. These are the predefined rules:
* **Product Group 1**: After iterative discussions with marketing, we think that there are some stock items with **high Margin** but exclude those that already have high contribution to Total Revenue. 
* **Product Group 2**: As part of the experimentation, we want to also understand how these stock items would perform against those with **low Margin** but exclude those that already have high contribution to Total Revenue. 

Once we identify these groups, we will conclude with recommendation and potential next steps for experimentation especially what to track.

## Step 1 - Get and inspect the data

To access the Wide World Importers database:

| Server | Database | Credential |
| --- | --- | --- |
| 10.91.134.93,11433 | WideWorldImporters | Windows Auth

To get a copy of your own WideWorldImporters database, go to: https://github.com/Microsoft/sql-server-samples/releases/tag/wide-world-importers-v1.0 

Here's the assumption / description about the data that will be used in the analysis today.
* For the purpose of analysis prepared for Feb 2016 meeting, we will use the latest month data, i.e. Jan 2016. It is possible to extend this to show month over month trend.
* We will use Sales.Invoices and Sales.InvoiceLines which has information about Revenue for each invoice line item (i.e. ExtendedPrice) and Profit (i.e. LineProfit).
* Warehouse.Stock* tables have more information about the stock items. 
* No cumulative analysis needed for this document. 

> Note: A stock item can belong to many different stock groups. So it is important that throughout the analysis, when / if we want to do cumulative total or percentage of cumulative total, that we choose only one group that it belongs to "consistently".





### Show Top 10 rows from the tables that we will be using today. 

In [22]:
USE WideWorldImporters;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160201';
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

SELECT TOP 10
    --  COUNT(DISTINCT g.StockItemStockGroupID) StockGroups
     i.InvoiceDate
    ,i.InvoiceID
    ,si.StockItemName
    ,gr.StockGroupName
    ,il.ExtendedPrice AS Revenue
    ,il.LineProfit AS Profit
    ,il.Quantity
    ,il.LineProfit * 100.0 / il.ExtendedPrice AS [Margin%]    
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= '20160101'
AND  i.InvoiceDate < '20160201'
ORDER BY i.InvoiceID, i.InvoiceDate, si.StockItemName

Commands completed successfully.

Total execution time: 00:00:00.103

InvoiceDate,InvoiceID,StockItemName,StockGroupName,Revenue,Profit,Quantity,Margin%
2016-01-01,61321,"""The Gu"" red shirt XML tag t-shirt (White) XXS",Clothing,1490.40,792.00,72,53.1400966183574879
2016-01-01,61321,"""The Gu"" red shirt XML tag t-shirt (White) XXS",T-Shirts,1490.40,792.00,72,53.1400966183574879
2016-01-01,61321,"""The Gu"" red shirt XML tag t-shirt (White) XXS",Computing Novelties,1490.40,792.00,72,53.1400966183574879
2016-01-01,61322,Tape dispenser (Red),Packaging Materials,1840.00,750.00,50,40.7608695652173913
2016-01-01,61323,"""The Gu"" red shirt XML tag t-shirt (White) XS",Clothing,1242.00,660.00,60,53.1400966183574879
2016-01-01,61323,"""The Gu"" red shirt XML tag t-shirt (White) XS",T-Shirts,1242.00,660.00,60,53.1400966183574879
2016-01-01,61323,"""The Gu"" red shirt XML tag t-shirt (White) XS",Computing Novelties,1242.00,660.00,60,53.1400966183574879
2016-01-01,61324,Black and orange glass with care despatch tape 48mmx75m,Packaging Materials,510.60,252.00,120,49.3537015276145710
2016-01-01,61325,Shipping carton (Brown) 305x305x305mm,Packaging Materials,503.13,200.00,125,39.7511577524695406
2016-01-01,61326,"""The Gu"" red shirt XML tag t-shirt (White) 5XL",Clothing,745.20,360.00,36,48.3091787439613526


Show the number of unique stock items and unique stock item groups sold in January

In [23]:
USE WideWorldImporters;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160201'; 
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

SELECT 
     COUNT(DISTINCT si.StockItemName) UniqueStockItem
    ,COUNT(DISTINCT gr.StockGroupName) UniqueStockGroup
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate;

Commands completed successfully.

Total execution time: 00:00:00.146

UniqueStockItem,UniqueStockGroup
227,9


## Step 2 - Identify Product Group 1

**Product Group 1**: After iterative discussions with marketing, we think that there are some stock items with **high Margin** but low contribution to Total Revenue. 

High margin refers to those that have higher than the Margin Percentage of all invoices in Jan 2016. 

In [24]:
USE WideWorldImporters;

SET NOCOUNT ON;

DROP TABLE IF EXISTS #ProductGroup1;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160201'; 
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

DECLARE @TotalRevenue AS DECIMAL(18, 2);
DECLARE @TotalProfit AS DECIMAL(18, 2);
DECLARE @MarginPct AS DECIMAL(18,2);
DECLARE @LowRevenueContribution AS DECIMAL(18,2) = 10;

SELECT
     @TotalRevenue  = SUM(il.ExtendedPrice)
    ,@TotalProfit   = SUM(il.LineProfit)
    ,@MarginPct     = SUM(il.LineProfit) * 100 / SUM(il.ExtendedPrice)
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate;

SELECT 
     @MarginPct AS [% Margin Threshold]
    ,@LowRevenueContribution AS [% Low Revenue Threshold];


SELECT 
    --  COUNT(DISTINCT g.StockItemStockGroupID) StockGroups
     gr.StockGroupName
    ,SUM(il.ExtendedPrice) Revenue
    ,SUM(il.LineProfit) Profit
    ,SUM(il.Quantity) QuantitySold
    ,SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) AS [Margin%]
    ,SUM(il.ExtendedPrice) * 100.0 / @TotalRevenue AS [%TotalRevenue]
    -- ,SUM(il.LineProfit) * 100.0 / @TotalProfit AS [%TotalProfit]
    ,COUNT(DISTINCT si.StockItemName) StockItems
INTO #ProductGroup1
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate
GROUP BY gr.StockGroupName
HAVING SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) > @MarginPct
AND SUM(il.ExtendedPrice) * 100.0 / @TotalRevenue < @LowRevenueContribution;

SELECT * FROM #ProductGroup1
ORDER BY Revenue DESC;

SELECT DISTINCT
    pg.StockGroupName
    ,si.StockItemName
FROM Warehouse.StockItems si
INNER JOIN Warehouse.StockItemStockGroups sg
      ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
    INNER JOIN #ProductGroup1 pg
        ON pg.StockGroupName = gr.StockGroupName
ORDER BY  
     pg.StockGroupName
    ,si.StockItemName;


Commands completed successfully.

Total execution time: 00:00:00.155

% Margin Threshold,% Low Revenue Threshold
42.98,10.00


StockGroupName,Revenue,Profit,QuantitySold,Margin%,%TotalRevenue,StockItems
Furry Footwear,116462.80,74418.50,7316,63.898944,2.533708,24
Mugs,81567.20,46376.00,5456,56.856187,1.774536,42


StockGroupName,StockItemName
Furry Footwear,Animal with big feet slippers (Brown) L
Furry Footwear,Animal with big feet slippers (Brown) M
Furry Footwear,Animal with big feet slippers (Brown) S
Furry Footwear,Animal with big feet slippers (Brown) XL
Furry Footwear,Dinosaur battery-powered slippers (Green) L
Furry Footwear,Dinosaur battery-powered slippers (Green) M
Furry Footwear,Dinosaur battery-powered slippers (Green) S
Furry Footwear,Dinosaur battery-powered slippers (Green) XL
Furry Footwear,Furry animal socks (Pink) L
Furry Footwear,Furry animal socks (Pink) M


## Step 3 - Identify Product Group 2

**Product Group 2**: As part of the experimentation, we want to also understand how these stock items would perform against those with **low margin** and low contribution to Total Revenue. 

High margin refers to those that have higher than the Margin Percentage of all invoices in Jan 2016. 

In [25]:
USE WideWorldImporters;
SET NOCOUNT ON;

DROP TABLE IF EXISTS #ProductGroup2;

-- Must use the same start and end date for the analysis in this paper. 
DECLARE @StartDate DATETIME = '20160201'; 
DECLARE @EndDate DATETIME = DATEADD(MONTH, 1, @StartDate);

DECLARE @TotalRevenue AS DECIMAL(18, 2);
DECLARE @TotalProfit AS DECIMAL(18, 2);
DECLARE @MarginPct AS DECIMAL(18,2);
DECLARE @LowRevenueContribution AS DECIMAL(18,2) = 10;

SELECT
     @TotalRevenue  = SUM(il.ExtendedPrice)
    ,@TotalProfit   = SUM(il.LineProfit)
    ,@MarginPct     = SUM(il.LineProfit) * 100 / SUM(il.ExtendedPrice)
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate;

SELECT 
     @MarginPct AS [% Margin Threshold]
    ,@LowRevenueContribution AS [% Low Revenue Threshold];


SELECT 
    --  COUNT(DISTINCT g.StockItemStockGroupID) StockGroups
     gr.StockGroupName
    ,SUM(il.ExtendedPrice) Revenue
    ,SUM(il.LineProfit) Profit
    ,SUM(il.Quantity) QuantitySold
    ,SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) AS [Margin%]
    ,SUM(il.ExtendedPrice) * 100.0 / @TotalRevenue AS [%TotalRevenue]
    -- ,SUM(il.LineProfit) * 100.0 / @TotalProfit AS [%TotalProfit]
    ,COUNT(DISTINCT si.StockItemName) StockItems
INTO #ProductGroup2
FROM Sales.InvoiceLines il
    INNER JOIN Sales.Invoices i
        ON i.InvoiceID = il.InvoiceID
    INNER JOIN Warehouse.StockItems_Archive si
        ON si.StockItemID = il.StockItemID 
        AND i.InvoiceDate BETWEEN si.ValidFrom AND si.ValidTo    
    INNER JOIN Warehouse.StockItemStockGroups sg
        ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
        AND i.InvoiceDate BETWEEN gr.ValidFrom AND gr.ValidTo
WHERE i.InvoiceDate >= @StartDate
AND  i.InvoiceDate < @EndDate
GROUP BY gr.StockGroupName
HAVING SUM(il.LineProfit) * 100.0 / SUM(il.ExtendedPrice) < @MarginPct
AND SUM(il.ExtendedPrice) * 100.0 / @TotalRevenue < @LowRevenueContribution;

SELECT * FROM #ProductGroup2
ORDER BY Revenue DESC;

SELECT 
    b.StockGroupName
    ,si.StockItemName
FROM Warehouse.StockItems_Archive si
INNER JOIN Warehouse.StockItemStockGroups sg
      ON sg.StockItemID = si.StockItemID        
    INNER JOIN Warehouse.StockGroups gr
        ON gr.StockGroupID = sg.StockGroupID
    INNER JOIN #ProductGroup2 b
        ON b.StockGroupName = gr.StockGroupName;


Commands completed successfully.

Total execution time: 00:00:00.161

% Margin Threshold,% Low Revenue Threshold
42.98,10.00


StockGroupName,Revenue,Profit,QuantitySold,Margin%,%TotalRevenue,StockItems
Toys,405000.10,145838.50,2865,36.009497,8.810987,21
USB Novelties,84704.77,31791.80,1877,37.532478,1.842796,14


StockGroupName,StockItemName
Toys,Pack of 12 action figures (female)
Toys,Pack of 12 action figures (male)
Toys,Pack of 12 action figures (variety)
Toys,Ride on big wheel monster truck (Black) 1/12 scale
Toys,Ride on vintage American toy coupe (Black) 1/12 scale
Toys,Ride on vintage American toy coupe (Red) 1/12 scale
Toys,Ride on toy sedan car (Pink) 1/12 scale
Toys,Ride on toy sedan car (Yellow) 1/12 scale
Toys,Ride on toy sedan car (Green) 1/12 scale
Toys,Ride on toy sedan car (Blue) 1/12 scale


## Conclusion

Out of the 9 stock groups sold in January 2016, these are the proposed two groups that we can do marketing experimentation with:
1. **Product Group 1** - Furry Footwear, USB Novelties and Mugs with  percentage of Total Revenue between 2.53% and 1.77%. 
2. **Product Group 2** - Clothing, Novelty Items and Toys with percentaage of Total Revenue between 8.81% and 1.84%

This paper includes their January baseline of the Quntity Sold, the Profit and the Revenue. The next analysis should include these metrics including the % month over month growth for the treatment group and the control group of each Product Group. 



In [31]:
SELECT 
    'PG1:' + StockGroupName AS StockGroup
    ,[Margin%]
FROM #ProductGroup1
UNION ALL 
SELECT    
    'PG2:' + StockGroupName AS StockGroup
    ,[Margin%]
FROM #ProductGroup2

Commands completed successfully.

Total execution time: 00:00:00.010

StockGroup,Margin%
PG1:Furry Footwear,63.898944
PG1:Mugs,56.856187
PG2:USB Novelties,37.532478
PG2:Toys,36.009497
